In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

import torch
import csv
import pandas as pd

device = 'cuda' if torch.cuda.is_available else 'cpu'
print(device)

cuda


In [9]:
file_path = 'runs/16/dialogpt/real/cornell_movie_dataset_5fts.csv'
output_path = 'runs/16/dialogpt/real/cornell_movie_dataset_5fts_gen.csv'

In [10]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large").to(device=device)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

ImportError: 
AutoModelForCausalLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.


In [ ]:
df = pd.read_csv(file_path)
f = open(output_path, 'a', encoding="utf-8")
writer = csv.writer(f)
writer.writerow(['aave_gen', 'sae_gen'])
# drop_idx = []

for i in tqdm(range(len(df['history']))):
    chat_history_sae_ids = tokenizer.encode(df.iloc[i]['history'], return_tensors='pt').to(device=device)
    chat_history_aave_ids = tokenizer.encode(df.iloc[i]['history_aave'], return_tensors='pt').to(device=device)
    # if chat_history_sae_ids.shape[1] >= 1000 or chat_history_aave_ids.shape[1] >= 1000:
    #     drop_idx.append(i)
    #     continue
    output_aave_ids = model.generate(chat_history_aave_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    output_sae_ids = model.generate(chat_history_sae_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    writer.writerow([
        tokenizer.decode(output_aave_ids[:, chat_history_aave_ids.shape[-1]:][0], skip_special_tokens=True), 
        tokenizer.decode(output_sae_ids[:, chat_history_sae_ids.shape[-1]:][0], skip_special_tokens=True)
    ])

f.close()